In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [25]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [26]:
import re
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.ml.feature import HashingTF, IDF, StopWordsRemover, Normalizer
from pyspark.sql.functions import col

In [27]:
spark

In [28]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, col, isnan, isnull, broadcast, desc, lower
from pyspark.sql.types import FloatType, ArrayType, StringType
import json
import re

In [29]:
var10 = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

In [30]:
var10

[[23126, 'en', 'Compass - powerful SASS library that makes your life easier'],
 [21617, 'en', 'Preparing for the AP* Computer Science A Exam — Part 2'],
 [16627, 'es', 'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'],
 [11556,
  'es',
  'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'],
 [16704, 'ru', 'Программирование на Lazarus'],
 [13702, 'ru', 'Математическая экономика']]

In [31]:
var10_ids = sorted([x[0] for x in var10])

In [32]:
var10_ids

[11556, 13702, 16627, 16704, 21617, 23126]

In [33]:
!hdfs dfs -ls /labs/slaba02/D

ls: `/labs/slaba02/D': No such file or directory


In [34]:
courses = spark.read.json("/labs/slaba02/DO_record_per_line.json")

In [35]:
courses = courses.withColumn("desc", lower(col("desc")))

In [36]:
courses

DataFrame[cat: string, desc: string, id: bigint, lang: string, name: string, provider: string]

In [37]:
courses.show(10)

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|this course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|this online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|this course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|we live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|this self-paced c...|  8|  en|           Bioethics|Canvas Network|
|9/humanities|15/m...|this game-based c...|  9|  en|College Foundatio...|Canvas Network|
|  14/social_sciences|what’s in your di...| 10|  en|Digital Literacies I|Canvas Network|
|  14/social_sciences|the goal of the d...| 11|  en|Digital Literacie...|Canvas Network|
|  14/social_sciences

In [38]:
temp_data = courses.select("desc")
temp_data.show()

+--------------------+
|                desc|
+--------------------+
|this course intro...|
|this online cours...|
|this course is ta...|
|we live in a digi...|
|this self-paced c...|
|this game-based c...|
|what’s in your di...|
|the goal of the d...|
|ready to explore ...|
|this self-paced c...|
|what is “interpro...|
|this course prese...|
|chemistry is an i...|
|are you consideri...|
|princess stories ...|
|this first instal...|
|this course exami...|
|this course will ...|
|the field of tech...|
|are you a higher ...|
+--------------------+
only showing top 20 rows



In [39]:
tokenizer = Tokenizer(inputCol="desc", outputCol="desc_words")

In [40]:
stopwords = StopWordsRemover.loadDefaultStopWords("english")

In [41]:
StopWordsRemover = StopWordsRemover(inputCol="desc_words", outputCol="words_filtered", stopWords=stopwords)
tf = HashingTF(inputCol="words_filtered", outputCol="words_hashed", numFeatures=10000) 
idf = IDF(inputCol="words_hashed", outputCol="features", minDocFreq=5)  

In [42]:
Pipeline1 = Pipeline(stages = [tokenizer,StopWordsRemover,tf, idf])

In [43]:
dataset = Pipeline1.fit(courses).transform(courses)
dataset = dataset.select("id", "lang", "name", "words_filtered", "features")
dataset.show(5, True)

+---+----+--------------------+--------------------+--------------------+
| id|lang|                name|      words_filtered|            features|
+---+----+--------------------+--------------------+--------------------+
|  4|  en|Accounting Cycle:...|[course, introduc...|(10000,[36,42,63,...|
|  5|  en|American Counter ...|[online, course, ...|(10000,[32,222,29...|
|  6|  fr|Arithmétique: en ...|[course, taught, ...|(10000,[30,41,246...|
|  7|  en|Becoming a Dynami...|[live, digitally,...|(10000,[493,721,8...|
|  8|  en|           Bioethics|[self-paced, cour...|(10000,[32,65,115...|
+---+----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [44]:
#Выбираю мои курсы
search_courses_df = dataset.where(col("id").isin(var10_ids))

#Переименовываю данные для джойна
search_courses_idf = search_courses_df.select("id", "lang", "name", "features") \
    .withColumnRenamed("id", "search_id") \
    .withColumnRenamed("lang", "search_lang") \
    .withColumnRenamed("name", "search_name") \
    .withColumnRenamed("features", "search_features") \


joined_data = dataset.join(broadcast(search_courses_idf), dataset.lang == search_courses_idf.search_lang) \
    .filter("lang = search_lang") \
    .filter("id != search_id")  



In [45]:
# User-defined function для косинусного расстояния
from pyspark.sql.functions import udf

@udf
def cos_sim(v1,v2):
    try:
        return float(v1.dot(v2))
    except:
        return 0




In [46]:
# рассчиттываю косинусное расстояние между моими курсами и всеми остальными
joined_data = joined_data.withColumn("cos_sim", cos_sim(col("features"), col("search_features"))) \
                .filter((isnan(col("cos_sim")) == False) & (isnull(col("cos_sim")) == False))

joined_data.cache()



DataFrame[id: bigint, lang: string, name: string, words_filtered: array<string>, features: vector, search_id: bigint, search_lang: string, search_name: string, search_features: vector, cos_sim: string]

In [47]:
# Собираю все рекомендации в словарь
result = dict()
for course_id in var10_ids:
    res = joined_data.filter(col("search_id") == course_id).orderBy(desc("cos_sim"), "name", "id").limit(10).select("id").collect()
    ids = [row[0] for row in res]
    result[str(course_id)] = ids

In [48]:
result

{'11556': [13152, 24706, 12308, 24162, 5936, 20157, 4255, 8952, 9471, 22797],
 '13702': [860, 13701, 1052, 17128, 5221, 25829, 1420, 10735, 9240, 17019],
 '16627': [16694, 12166, 13438, 4100, 7818, 22758, 25563, 10034, 18863, 24221],
 '16704': [17071, 7174, 13479, 8081, 1187, 10985, 8783, 1237, 8202, 1200],
 '21617': [21666, 21840, 21523, 8289, 21614, 24469, 12790, 11067, 22637, 5478],
 '23126': [17847, 6549, 9823, 18907, 24140, 13343, 13130, 22298, 4803, 5944]}

In [49]:
#save data 
with open('lab02.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=4)

In [50]:
#check result
!cat lab02.json

{
    "11556": [
        13152,
        24706,
        12308,
        24162,
        5936,
        20157,
        4255,
        8952,
        9471,
        22797
    ],
    "13702": [
        860,
        13701,
        1052,
        17128,
        5221,
        25829,
        1420,
        10735,
        9240,
        17019
    ],
    "16627": [
        16694,
        12166,
        13438,
        4100,
        7818,
        22758,
        25563,
        10034,
        18863,
        24221
    ],
    "16704": [
        17071,
        7174,
        13479,
        8081,
        1187,
        10985,
        8783,
        1237,
        8202,
        1200
    ],
    "21617": [
        21666,
        21840,
        21523,
        8289,
        21614,
        24469,
        12790,
        11067,
        22637,
        5478
    ],
    "23126": [
        17847,
        6549,
        9823,
        18907,
        24140,
        13343,
        13130,
        22298,
        4803,
        5944
    

In [51]:
sc.stop()